# 🛠️ Housing Preprocessing

This notebook handles the preprocessing phase of a housing dataset aimed at predicting property prices. It includes cleaning steps such as feature selection, feature engineering, target reordering, and data scaling. These transformations prepare the dataset for training robust and effective regression models.


In [35]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit

In [5]:
#🔍 Load the dataset
file = 'real_estate_dataset.csv'
df = pd.read_csv(file)
df.head()

,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,2,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,3,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,4,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,5,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523


In [6]:
# Dropping the 'ID' column since it doesn't provide predictive power
df = df.drop(columns=['ID'])

In [7]:
# Creating new features: price per square feet, decade of construction
df['price_per_ft2'] = df['Price'] / df['Square_Feet']
df['Decade'] = (df['Year_Built'] // 10) * 10

In [8]:
# Selecionar colunas contínuas (ajuste conforme seu df)
continuous_cols = [
    "Square_Feet", "Garage_Size", "Location_Score", 
    "Distance_to_Center", "Price", "price_per_ft2"
]

outlier_summary = []

for col in continuous_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detecting outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_pct = 100 * len(outliers) / len(df)

    outlier_summary.append({
        "Variable": col,
        "Num_Outliers": len(outliers),
        "Percent_Outliers": round(outlier_pct, 2)
    })

outlier_df = pd.DataFrame(outlier_summary)
outlier_df.sort_values(by="Percent_Outliers", ascending=False, inplace=True)
outlier_df.reset_index(drop=True, inplace=True)

outlier_df


,Variable,Num_Outliers,Percent_Outliers
0,price_per_ft2,32,6.4
1,Price,1,0.2
2,Square_Feet,0,0.0
3,Garage_Size,0,0.0
4,Location_Score,0,0.0
5,Distance_to_Center,0,0.0


The provided outlier summary shows that `price_per_ft2` has 32 outliers (6.4%), a significant enough portion to warrant treatment.
Price has only 1 outlier (0.2%), which is negligible and can likely be ignored.Other variables (`Square_Feet`, `Garage_Size`, etc.) show no outliers.
Aplying Log Transformation preserves Data ulike removal, log-transform retains all records while reducing skewness.


In [9]:
# Apply log transformation to price_per_ft2
df['price_per_ft2_log'] = np.log(df['price_per_ft2'] + 1)  # +1 to avoid log(0)

We will keep both the original price_per_ft2 column and the log-transformed version in the dataset. During model development, we will evaluate which version performs better and use the most suitable one for the final model implementation.

In [10]:
# Remove outlier in Price based on IQR
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1
upper_limit = Q3 + 1.5 * IQR

df = df[df['Price'] <= upper_limit]

We applied a log transformation to the price_per_ft2 variable to reduce the effect of extreme values and improve distribution symmetry. For the Price variable, we identified outliers using the Interquartile Range (IQR) method and removed records exceeding the upper limit to ensure data quality and model robustness.

In [11]:
# Create a copy to avoid modifying the original dataset
df_strat = df.copy()
# Initialize the stratified splitter (80% train, 20% test)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [13]:
# Perform the split
for train_idx, test_idx in split.split(df_strat, df_strat["Num_Bedrooms"]):
    strat_train_set = df_strat.iloc[train_idx]
    strat_test_set = df_strat.iloc[test_idx]

In [14]:
# Drop target variable and store separately
X_train = strat_train_set.drop("Price", axis=1)
y_train = strat_train_set["Price"].copy()

X_test = strat_test_set.drop("Price", axis=1)
y_test = strat_test_set["Price"].copy()

In [15]:
# Function to calculate the proportions of Num_Bedrooms values
def bedrooms_proportions(data):
    return data["Num_Bedrooms"].value_counts() / len(data)

# Random split for comparison
train_set_random, test_set_random = train_test_split(df, test_size=0.2, random_state=42)

# Create a DataFrame comparing proportions
compare_props = pd.DataFrame({
    "Overall": bedrooms_proportions(df),
    "Stratified": bedrooms_proportions(X_train),
    "Random": bedrooms_proportions(train_set_random),
}).sort_index()

# Calculate percentage errors
compare_props["Random %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Stratified %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

# Display result
compare_props

,Overall,Stratified,Random,Random %error,Stratified %error
Num_Bedrooms,,,,,
1,0.218437,0.218045,0.218045,-0.179347,-0.179347
2,0.194389,0.195489,0.197995,1.855153,0.565848
3,0.210421,0.210526,0.210526,0.050125,0.050125
4,0.168337,0.167920,0.162907,-3.225325,-0.247643
5,0.208417,0.208020,0.210526,1.012146,-0.190380


We chose the `Num_Bedrooms` column for stratified sampling because it is the feature with the highest correlation to the target variable (`Price`). Additionally, using `Num_Bedrooms` is a less arbitrary choice compared to a continuous variable like `Square_Feet`, as it is a discrete variable with a manageable number of distinct categories. This makes it well-suited for creating representative training and testing subsets while preserving the distribution of key housing characteristics.

In [16]:
#Standardize the features
scaler = MinMaxScaler()

# Fit on the training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#convert back to DataFrame 
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [17]:
X_train_scaled.describe()

,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,price_per_ft2,Decade,price_per_ft2_log
count,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000
mean,0.482244,0.488095,0.487469,0.483709,0.471596,0.526316,0.473684,0.514748,0.534010,0.531973,0.277037,0.436090,0.440400
std,0.301724,0.359739,0.407028,0.403017,0.291833,0.499934,0.499934,0.300895,0.285740,0.282013,0.218077,0.291302,0.231634
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.216522,0.250000,0.000000,0.000000,0.220833,0.000000,0.000000,0.256410,0.312261,0.311749,0.114671,0.166667,0.259522
50%,0.474856,0.500000,0.500000,0.500000,0.475000,1.000000,0.000000,0.512821,0.547834,0.549794,0.202098,0.416667,0.397338
75%,0.758069,0.750000,1.000000,1.000000,0.716667,1.000000,1.000000,0.794872,0.782020,0.766268,0.389372,0.666667,0.609842
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


We chose `MinMax Scaler` because it scales data to a positive range (0-1), crucial for gradient stability. Unlike `StandardScaler`, which can create negative values, MinMax ensures all features remain positive, benefiting activation functions and data with inherent positive bounds. This promotes more effective and stable for neural network training.

We have completed the **feature engineering** stage.  
In the next notebook, **Modeling**, we will implement the transformations performed in this stage using a **pipeline**, test different **machine learning models**, and apply the necessary adjustments to optimize performance.